# IMPORT

In [ ]:
import tempfile
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

# LOAD DATA, TRAIN MODEL

In [ ]:
%%bash

if [ ! -d ./data ]; then
    mkdir ./data
fi

if [ ! -f ./data/trainingandtestdata.zip ]; then
    wget -q -O ./data/trainingandtestdata.zip http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
fi

unzip -n ./data/trainingandtestdata.zip -d ./data

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
df_train_file_path = './data/training.1600000.processed.noemoticon.csv'
df_train_name = 'training.1600000.processed.noemoticon'
df_train = pd.read_csv(df_train_file_path,
                      header=None,
                      encoding='ISO-8859-1')

df_test_file_path = './data/testdata.manual.2009.06.14.csv'
df_test_name = 'testdata.manual.2009.06.14'
df_test = pd.read_csv(df_test_file_path,
                     header=None,
                     encoding='ISO-8859-1')
df_train.columns = columns
df_test.columns = columns
df_train.shape[0]

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df=100,
                                                        ngram_range=(1,2),
                                                        stop_words='english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(df_train.text, df_train.polarity)

In [ ]:
x_test, y_test = df_test.text[df_test.polarity != 2], df_test.polarity[df_test.polarity != 2]
print(classification_report(y_test, sentiment_lr.predict(x_test)))

In [ ]:
sentiment_lr.predict([x_test[0]])

In [ ]:
sentiment_lr.predict(['good', 'bad'])

# UNBOX

In [4]:
import unboxapi
client = unboxapi.UnboxClient(email="me@vikasnair.com", password="00000000")

## Create function

In [ ]:
# class_dict = { 4: 'positive', 2: 'neutral', 0: 'negative'}
class_names = ['negative', 'positive']
dataset_labels = [0, 4]
def predict_function(model, text_list):
    return model.predict_proba(text_list), class_names, dataset_labels

In [ ]:
texts = ['some new text, sweet noodles', 'happy time', 'sad day']

predict_function(sentiment_lr, texts)

# Package (function, model) & Upload to Firebase

In [ ]:
print('Uploading model...')
client.add_model(function=predict_function, 
                 model=sentiment_lr, 
                 name='03.15.2021.sentiment_analyzer',
                 description='this is my sklearn sentiment model')
print('Complete.')

In [5]:
print('\nUploading dataset (from file)...')
with tempfile.TemporaryDirectory() as tmpdir:
    df_train.to_csv(f'{tmpdir}/dataset.csv')
    response_i = client.add_dataset(file_path=f'{tmpdir}/dataset.csv', 
                                    name=df_train_name, 
                                    description='this is my sentiment train dataset',
                                    label_column_name='polarity',
                                    text_column_name='text')
print(f'Complete. Response: {response_i}')


Uploading dataset (from file)...
Complete. Response: {'columns': None, 'datasetId': 'e38d238c-876f-11eb-a2eb-1e008a21a079', 'description': 'this is my sentiment train dataset', 'labelColumnName': 'polarity', 'name': 'training.1600000.processed.noemoticon', 'textColumnName': 'text'}


In [ ]:
print('\nUploading dataset (from data frame)...')
response_j = client.add_dataframe(df=df_test, 
                                  name=df_test_name, 
                                  description='this is my sentiment test dataset',
                                  label_column_name='polarity',
                                  text_column_name='text')
print(f'Complete. Response: {response_j}')

In [ ]:
# Test error analysis / dataset-model association
result = client._test_associate(model_id='', 
                                dataset_id='62f26430-845e-11eb-a9b2-1e008a21a079',
                                user_id='ytGD2XvoGPSaippqWhAmi5V8mHT2')
print(result.content)

In [ ]:
# Test checklist invariance test
test_config = {'testType': 'inv', 'examplesPerRow': 3, 'perturbationMode': 'change_names'}

result = client._test_add_test_suite(run_id='', 
                                     user_id='ytGD2XvoGPSaippqWhAmi5V8mHT2',
                                     test_config=test_config)
print(result.content)